# DO THIS :  

### *** Names: [Insert Your Names Here;      Optional: indicate preferred pronouns]***

# AND IN THE FILENAME

##### Problem Set 11             
## PHYS 105A   Spring 2019

## Contents

Review of some Python syntax and concepts

** There are 6 exercises that must be completed. **

### Setting up jupyter-notebook for running animations


#### To produce the animation in a separate window use
   %matplotlib tk  
   
as the first line. Graphics will be plotted in separate windows from the web browser
where the notebook is displayed.

To terminate an animation running in a separate window, kill the window by clicking on its "close" button.

This way of displaying animations works with both animation styles from a jupyter-notebook.

#### To produce the animation embedded within the notebook, use
%matplotlib notebook  

as the first line. 

To terminate an animation running within a notebook, click on the "power" button at the right end of the title line.

To use this form of animation, you must use the  `FuncAnimation` style of animation described below.

**NB**: You will need to restart the kernel when you change the backend.  
For example, from the "Kernel" menu at the top of the notebook, choose "Restart & Clear Output"

**NB**: Animations run much faster in a separate window, so choose this option if you have a  
complex animation which runs too slowly when embedded in the notebook.


To start, as usual we import matplotlib and numpy.

This time, however, we first need to specify where animations will be displayed (descibed above) and to import the
`FuncAnimation` function from the `matplotlib.animation` module

In [ ]:

%matplotlib tk
# on windows . %matplotlib qt 
# different back end.


import numpy as np
from matplotlib import pyplot as plt

# additional modules for animation:
from matplotlib.animation import FuncAnimation

# import matplotlib so we can use dir on it below
import matplotlib

We have already seen how to keep an instance of a matplotlib object like axes. We can also keep an instance of an object within the plot. For example,

In [ ]:
x = np.linspace(-10,10,400)
y = np.sin(x)

fig, ax = plt.subplots()
line = ax.plot(x,y)

plt.plot returns a list of objects corresponding to each line plotted. If we look at the value of line

In [ ]:
print(line)

we see that it is a list with one element: a Line2D object. Let's have a look at the member functions available for this object

In [ ]:
dir(matplotlib.lines.Line2D)

If you scroll down, you will see that there are a bunch of methods beginning with "set_". The one we are interested in here is the set_data function. This allows us to set the data used by the Line2D object.

Let's try setting the data used for the line. We'll use the same x values, but use y values which are half the previous
ones:

In [ ]:
line[0].set_data(x, 0.5*y)

If we continuously reset this data, we can create the appearance of a moving line -- an animation

In [ ]:
for eps in np.arange(0,2*np.pi,0.01):
    line[0].set_data(x, np.sin(x+eps))

That didn't seem to do much!

The trouble is that our animation went by faster than the eye could see. We need to pause and display each step in the animation to give our eye time to register the changed plot. We can do this with the plt.pause() function, with an argument given the amount of time (in seconds) to pause before continuing

In [ ]:
x = np.linspace(-10,10,400)
y = np.sin(x)

fig, ax = plt.subplots()
line = ax.plot(x,y)

for eps in np.arange(0,10*np.pi,0.1):
    line[0].set_data(x, np.sin(x+eps))
    plt.pause(0.01)
plt.show()

There are equivalent "set_" methods for most of the plot objects. So, for example, we could keep track of the value of eps being used in the plot as follows

In [ ]:
x = np.linspace(-10,10,400)
y = np.sin(x)

fig, ax = plt.subplots()
line = ax.plot(x,y)
epstext = ax.text(8,1.2, 'eps=0')

for eps in np.arange(0,10*np.pi,0.1):
    line[0].set_data(x, np.sin(x+eps))
    epstext.set_text(f"eps={eps:.2f}")
    plt.pause(0.01)
plt.show()

Let's practice with this by performing a simple simulation of balls moving in a box. We'll put all the balls at the origin to begin with, and then have each take a step of length $d=0.1$ in a random direction

In [ ]:
nballs = 100
d = 0.1
xpos = np.zeros(nballs)                # positions of the balls
ypos = np.zeros(nballs)

fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)

balls = ax.plot(x,y,'r.')              # plot object with balls
step = 0
steptext = ax.text(4,5.5,'step = 0')   # plot object with text for step number

for step in range(1,300):
    theta = np.random.random(nballs) * 2 * np.pi  # change balls' coordinates by d in a random direction
    xpos += d * np.cos(theta)
    ypos += d * np.sin(theta)
    
    balls[0].set_data(xpos, ypos)                 # change data in plot
    
    steptext.set_text(f"step = {step:3d}")

    plt.pause(0.01)                               # pause between frames

This is called  a"random walk" and is a model for diffusion. The RMS (root mean square) distance each ball should get from the origin after $N$ steps is $d\sqrt{N}$. 

The *average* distance is approximately $\sqrt{\frac{2}{\pi}}d\sqrt{N}$. 


Let's draw a circle of this radius at each timestep on top of our distribution of balls.

To draw a circle, we could just create a bunch of coordinates and use plot to show them. Here, however, we will use one of matplotlib's patch objects. Patchs are goemetric shapes (circles, ellipses, polygons, etc.) available as matplotlb methods.

Thus,

In [ ]:
nballs = 1000
center = [0,0]
xpos = np.zeros(nballs) + center[0]
ypos = np.zeros(nballs) + center[1]
d = 0.1

fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_aspect(1.0)                    # make box size proportional to coordinates

balls = ax.plot(xpos,ypos,'r.')             # object to draw balls
step = 0                              # number of steps
steptext = ax.text(4,5.5,'step = 0')  # text of step counter display

circle = plt.Circle(center, 0, edgecolor='green', facecolor='None', zorder=10)     # object with circle
ax.add_patch(circle)                                                               # high zorder plots it on top

for step in range(1,300):
    theta = np.random.random(nballs) * 2 * np.pi
    xpos += d * np.cos(theta)
    ypos += d * np.sin(theta)
    balls[0].set_data(xpos, ypos)

    rmsDist = d * np.sqrt(step)
    circle.set_radius(rmsDist)

    steptext.set_text(f"step = {step:3d}")
    
    plt.pause(0.01)

Next, let's count the number of balls inside the circle and write the fraction of balls within the circle at the top of the animation to check whether we have the right radius for the circle.

We'll also define a function to add the random steps of length $d$ to the positions.

In [ ]:
def countEm(x, y, center, radius):              # function to count number of particles closer to center than radius
    ninside = ( (x-center[0])**2 + (y-center[1])**2 < radius**2).sum()
    return ninside

def addStep(x, y, d):                           # function to add a random step to the positions
    theta = np.random.random(nballs) * 2 * np.pi
    x += d * np.cos(theta)
    y += d * np.sin(theta)

    # Don't need a return as we are changing only the data within the x and y arrays!

nballs = 10000
center = [0,0]
xpos = np.zeros(nballs) + center[0]
ypos = np.zeros(nballs) + center[1]
d = 0.1

fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_aspect(1.0)                    # make box size proportional to coordinates

balls = ax.plot(xpos,ypos,'r,')             # object to draw balls
step = 0                              # number of steps
steptext = ax.text(4,5.5,'step = 0')  # text of step counter display
ntext = ax.text(-2, 5.5, '')          # text of fraction of balls inside circle

circle = plt.Circle(center, 0, edgecolor='green', facecolor='None', zorder=10)     # object with circle
ax.add_patch(circle)                                                               # high zorder plots it on top

for step in range(1,500):
    addStep(xpos, ypos, d)
    balls[0].set_data(xpos, ypos)

    rmsDist = d * np.sqrt(step)

    circle.set_radius(rmsDist*np.sqrt(2/np.pi))

    steptext.set_text(f"step = {step:3d}")
    
    if step%10==0:
        nin = countEm(xpos, ypos, center, rmsDist*np.sqrt(2/np.pi))
        ntext.set_text(f'nin/nballs = {nin/nballs:.3f}')
    
    plt.pause(0.01)

Matplotlib has another way of doing animations. I don't like it as much, but it is the only way to produce matplotlib animations within the jupyter-notebook so it is worth learning. It also does some things that the previous method doesn't which we will cover later.

To use it, we set up our plot as before, keeping handles for each of the plot objects we wish to change.

We then create a function which updates the plot when called. This must return all of the objects which are going to be updated.

This is then given as an argument to the function `FuncAnimation` which actually produces the animation. There are many tweaks one can make using `FuncAnimation`; here we are just using the simplest case.

Thus, adapting our example from above

In [ ]:
def addStep(x, y, d):
    theta = np.random.random(nballs) * 2 * np.pi
    x += d * np.cos(theta)
    y += d * np.sin(theta)

    # Don't need a return as we are changing only the data within the x and y arrays
    
nballs = 1000
center = [0,0]
xpos = np.zeros(nballs) + center[0]
ypos = np.zeros(nballs) + center[1]
d = 0.1

fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_aspect(1.0)                    # make box size proportional to coordinates

balls = ax.plot(xpos,ypos,'r.')             # object to draw balls
step = 0                              # number of steps
steptext = ax.text(4, 5.5,'step = 0')  # text of step counter display

circle = plt.Circle(center, 0, edgecolor='green', facecolor='None', zorder=10)     # object with circle
ax.add_patch(circle)                                                               # high zorder plots it on top

def update(step):
    addStep(xpos, ypos, d)
    balls[0].set_data(xpos, ypos)

    rmsDist = d * np.sqrt(step)

    circle.set_radius(rmsDist*np.sqrt(2/np.pi))

    steptext.set_text(f"step = {step:3d}")
    
    return balls[0], circle, steptext

anim = FuncAnimation(fig, update, interval=10, frames=500, repeat=False)

One advantage of using `FuncAnimation` is that we can easily save our animation to a file.

In [1]:
def addStep(x, y, d):
    theta = np.random.random(nballs) * 2 * np.pi
    x += d * np.cos(theta)
    y += d * np.sin(theta)

    # Don't need a return as we are changing only the data within the x and y arrays
    
nballs = 1000
center = [0,0]
xpos = np.zeros(nballs) + center[0]
ypos = np.zeros(nballs) + center[1]
d = 0.1

fig, ax = plt.subplots()
ax.set_xlim(-5,5)
ax.set_ylim(-5,5)
ax.set_aspect(1.0)                    # make box size proportional to coordinates

balls = ax.plot(xpos,ypos,'r.')             # object to draw balls
step = 0                              # number of steps
steptext = ax.text(4, 5.5,'step = 0')  # text of step counter display

circle = plt.Circle(center, 0, edgecolor='green', facecolor='None', zorder=10)     # object with circle
ax.add_patch(circle)                                                               # high zorder plots it on top

def update(step):
    addStep(xpos, ypos, d)
    balls[0].set_data(xpos, ypos)

    rmsDist = d * np.sqrt(step)

    circle.set_radius(rmsDist*np.sqrt(2/np.pi))

    steptext.set_text(f"step = {step:3d}")
    
    return balls[0], circle, steptext

anim = FuncAnimation(fig, update, interval=10, frames=500, repeat=False)  # save the returned object
anim.save('foo.mp4')

NameError: name 'np' is not defined

For the exercises, we will make another "balls in a box" animation, but this time with different particle dynamics.

We will give the balls not only positions but also velocities, and then move the particles according to their velocities.

The basic code to make the particles looks like this:

In [ ]:
nBalls = 10
boxWidth = 10
boxHeight = 10
vmax = 1

xpos = np.random.random(nBalls) * boxWidth
ypos = np.random.random(nBalls) * boxHeight
theta = 2*np.pi*np.random.random(nBalls)
xvel = vmax * np.cos(theta)
yvel = vmax * np.sin(theta)

#### Exercise 1

Using the code above, write a program to plot the positions of these points within a set of axes:

 * Use plt.subplots() to obtain figure and axes objects
 * set the limits and aspect ratio of the plot using the variables defined above
 * plot the positions of the points as red, filled circles

We can now animate this system by following the balls' trajectories along straight lines. 

Taking our time-step as $\Delta t = 0.1$, the update to the balls' positions is

    xpos += xvel*dt
    ypos += yvel*dt
 
#### Exercise 2

Define a function called advance which, given xpos, ypos, xvel, yvel, and dt, updates the positions of the particles. Then use it to advance the positions of the particles by one step

#### Exercise 3

Modify your function to take an additional argument, time, and advance time by dt as well as the positions.
Have the function then return this new value of time (remember, it is a scalar variable -- we need to return it to
get the new value).

Using this new definition, and using the random-walk examples above as templates, write a code to evolve the balls from time=0 to time=1. While you need to animate the balls, you don't need to write the time at the top of the plot.

Copy all of the code you need to make the answer stand-alone, i.e., to not depend upon previous cells for its function.


#### Exercise 4

Most of your balls will have moved out of the box by the end of your simulation.

Let's add some boundary conditions to our box. Specifically, let's make the balls bounce off of the sides of the box.

To do so, we will modify our advance function:
  * advance the x and y positions as before
  * then check to see if a position is outside the box:  
      if x<0 or x>boxWidth, then multiply the x velocity of that ball by by -1   
      if y<0 or y>boxWidth, then multiple the y velocity of that ball by -1
      
You can apply the boundary conditions by writing a `for` loop to check each particle in turn, or you
can do it all in two lines using `numpy`. We recommend `numpy` since, in an animation, your code needs to run as fast as it can to keep up with the display.

Hint: 
  * we can modify only some elements in a numpy array using a conditional expression as an index. If we have two arrays
a and b of the same length, we can write expressions like

    a[ b>0 ] *= -1
    
 Remember, to use more complex conditional expressions in numpy, you can use the logical operators `&` (and) and `|` (or),
but the expressions to either side of these operators must be included within parentheses.
            
Again, copy everything from the previous code cell to this cell to form a stand-alone cell.

Once you have things working, you might wish to set the maximum time to a larger value to better follow the evolution of the balls

#### Exercise 5

We can make things a bit more interesting by making the dynamics more complicated.

Try adding gravity to your simulation. The update to the y components of x and xv becomes

    y += yv*dt + 0.5*g*dt**2
    yv += g*dt
    
Let g=-0.5, and modify your advance function to take this acceleration into account

#### Exercise 6

Finally, let's try modifying our boundary conditions. Instead of using reflecting boundary conditions, we will use
*periodic* boundary conditions in the x direction.

Under periodic boundary conditions in x, a ball which passes a wall to the left or right appears immediately at the opposite wall. The x velocity remains unchanged, but the width of the box is added or subtracted from x as appropriate.

The obvious next step is to modify this code so that the balls collide with themselves. This is harder than it might seem at first, but not that hard. In fact, with this modification your code becomes a simple version of a *molecular dynamics* simulation. We will discuss an example of this in class.